In [ ]:
import sys
sys.path.insert(0, '..')

In [ ]:
from config.path_config import GraphPaths, PlotPaths, NodePairPaths
from config.constants import Constants

import csv
import random
import os
random.seed(42)

### Facebook sampled subgraphs

In [ ]:
import networkx as nx

In [ ]:
def common_neighborhood(graph_1, graph_2, pair):
    """
    :param graph1: networkx graph 1
    :param graph2: networkx graph 2
    :param pair: expected a tuple for pair of nodes
    """
    l1 = list(nx.all_neighbors(graph_1, pair[0]))
    l2 = list(nx.all_neighbors(graph_2, pair[1]))
    union = list(set().union(l1, l2))
    intersection = list(set(l1).intersection(l2))
    cn = len(intersection) / len(union)
    return cn

In [ ]:
def get_common_neighborhood_list(graph_1, graph_2, list_of_pairs):
    list_of_cn = []
    for pair in list_of_pairs:
        list_of_cn.append(common_neighborhood(graph_1, graph_2, pair))
    return list_of_cn

### Generate seed, random and biased pairs

In [ ]:
alpha_s = 0.5
alpha_c = 0.5
u_suffix = '-u'
v_suffix = '-v'

In [ ]:
fbG1 = nx.read_edgelist(GraphPaths.fb_subgraph.format('1', alpha_s, alpha_c), data=(('weight', float),))
fbG2 = nx.read_edgelist(GraphPaths.fb_subgraph.format('2', alpha_s, alpha_c), data=(('weight', float),))

print('nodes: ', fbG1.number_of_nodes(), ', edges: ', fbG1.number_of_edges())
print('nodes: ', fbG2.number_of_nodes(), ', edges: ', fbG2.number_of_edges())

In [ ]:
remove_node_suffix = lambda label : label[:-2]

In [ ]:
fbG1 = nx.relabel_nodes(fbG1, remove_node_suffix)
fbG2 = nx.relabel_nodes(fbG2, remove_node_suffix)

### Generating node pairs

In [ ]:
def random_walk(graph, path_length, alpha=0, rand=random.Random(), start=None, is_start_node_first_node=True):
    if start:
        path = [start]
    else:
      # Sampling is uniform w.r.t V, and not w.r.t E
      path = [rand.choice(list(graph.nodes()))]
    while len(path) < path_length:
        cur = path[-1]
        if len(graph[cur]) > 0:
            if rand.random() >= alpha:
                path.append(rand.choice(list(graph[cur])))
            else:
                path.append(path[0])
        else:
            break
    if is_start_node_first_node:
        return [tuple((start, node)) for node in path]
    else:
        return [tuple((node, start)) for node in path]

In [ ]:
def get_intersecting_nodes(graph1, graph2):
    nodes_graph1 = list(graph1.nodes())
    nodes_graph2 = list(graph2.nodes())
    intersection_nodes = list(set(nodes_graph1).intersection(nodes_graph2))
    print(len(intersection_nodes))
    return intersection_nodes

In [ ]:
def write_sampled_nodepairs(graph1, graph2, random_walk_path_len, path_seed, path_nonseed, path_combined, intersection_downsize_factor=0):
    
    with open(path_seed, 'w') as f_seed, open(path_nonseed, 'w') as f_nonseed, open(path_combined, 'w') as f_combined:
        writer_seed = csv.writer(f_seed, delimiter=' ')
        writer_nonseed = csv.writer(f_nonseed, delimiter=' ')
        writer_combined = csv.writer(f_combined, delimiter=' ')
        total_num_nodepairs = 0
        
        intersection_nodes = get_intersecting_nodes(graph1, graph2)
        if intersection_downsize_factor > 0:
            sampled_intersection_nodes_len = int(len(intersection_nodes)/intersection_downsize_factor)
            intersection_nodes = random.sample(intersection_nodes, sampled_intersection_nodes_len)
            print('Sampled intersection nodes size: ', len(intersection_nodes))
        for node in intersection_nodes:
            assert(node in graph1 and node in graph2)
            
            random_walk_pairs = random_walk(graph2, random_walk_path_len, start=node)
            random_walk_pairs.extend(random_walk(graph1, random_walk_path_len, start=node, is_start_node_first_node=False))

            for node1, node2 in random_walk_pairs:
                # Duplicates will need be handled when reading; networkx handles duplicate edges
                assert(node1 in graph1 and node2 in graph2)
                cn = common_neighborhood(graph1, graph2, tuple((node1, node2)))
                if cn > 0:
                    if node1 == node2:
                        writer_seed.writerow([node1+u_suffix, node2+v_suffix, cn])
                    else:
                        writer_nonseed.writerow([node1+u_suffix, node2+v_suffix, cn])
                    writer_combined.writerow([node1+u_suffix, node2+v_suffix, cn])
                    total_num_nodepairs += 1
    print(f'Wrote {total_num_nodepairs} node pairs combined')

In [ ]:
# for experiments
DEFAULT_WALK_LENGTH = 20
out_dir = os.path.dirname(NodePairPaths.fb_nodepairs.format(alpha_s, alpha_c, type='seed'))
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
write_sampled_nodepairs(fbG1, fbG2, DEFAULT_WALK_LENGTH, 
                        NodePairPaths.fb_nodepairs.format(alpha_s, alpha_c, type='seed'), 
                        NodePairPaths.fb_nodepairs.format(alpha_s, alpha_c, type='nonseed'), 
                        NodePairPaths.fb_nodepairs.format(alpha_s, alpha_c, type='combined'))